In [ ]:
######################################################################
#
# Import OCI Python SDK library
#
######################################################################

import oci
from oci.usage_api.models import Filter, Dimension

from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
######################################################################
#
# Establish Authentication with API key-based authentication
# Other methods include:
#   -Session token-based authentication
#   -Instance principal
#   -Resource principal
#
# More info @ https://docs.oracle.com/en-us/iaas/Content/API/Concepts/sdk_authentication_methods.htm
#
#  Default location of the config file is "~/.oci/config"
#
#  Config file structure:
#
#   [DEFAULT]
#   user= <OBTAINED FROM OCI CONSOLE USER PROFILE>                   
#   fingerprint= <OBTAINED FROM OCI CONSOLE USER PROFILE>
#   tenancy=<TENANCY ROOT COMPARTMENT OCID>
#   region=us-ashburn-1
#   key_file=./config_default.key
#
######################################################################

try:
    # check if env variables config file location, and profile exist and use them
    env_config_file = './config'
    env_config_section = oci.config.DEFAULT_PROFILE

    # check if file exist
    if env_config_file is None or env_config_section is None:
        print("*** OCI_CONFIG_FILE and OCI_CONFIG_PROFILE env variables not found, abort. ***")
        print("")
        raise SystemExit

    config = oci.config.from_file(env_config_file, env_config_section)
    oci.config.validate_config(config)

    # Generate signer object from data in the config file which will be used to sign all of the requests to the API.
    # User in config file must have the right permissions to access the resources the API retrieves
    signer = oci.signer.Signer(
        tenancy=config["tenancy"],
        user=config["user"],
        fingerprint=config["fingerprint"],
        private_key_file_location=config.get("key_file"),
        pass_phrase=oci.config.get_config_value_or_default(config, "pass_phrase"),
        private_key_content=config.get("key_content")
    )

except KeyError:
    print("* Key Error obtaining delegation_token_file")
    raise SystemExit

except Exception:
    raise

In [ ]:

######################################################################
#
# Create Usage API client object.
# All Usage API operations will originate from this object.
#
# Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/client/oci.usage_api.UsageapiClient.html#oci.usage_api.UsageapiClient
#
#####################################################################

print("\nConnecting to UsageAPI Service...")

usage_client = oci.usage_api.UsageapiClient(config, signer=signer)

print("\nConnectiion Succesful!")

In [ ]:

######################################################################
#
# Define parameters for the query
#
# Group by available dimensions:
#   “tagNamespace”, 
#   “tagKey”, 
#   “tagValue”, 
#   “service”, 
#   “skuName”, 
#   “skuPartNumber”, 
#   “unit”,
#   “compartmentName”, 
#   “compartmentPath”, 
#   “compartmentId”, 
#   “platform”, 
#   “region”, 
#   “logicalAd”, 
#   “resourceId”, 
#   “tenantId”, 
#   “tenantName”
#
# Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/models/oci.usage_api.models.RequestSummarizedUsagesDetails.html
#
######################################################################

# Define required query details
tenant_id = config["tenancy"] 
time_usage_started = datetime(year=2024, month=10, day=15)
time_usage_ended = datetime(year=2024, month=10, day=21)
granularity = 'DAILY'

# request summarized usages
try:
    # oci.usage_api.models.RequestSummarizedUsagesDetails
    requestSummarizedUsagesDetails = oci.usage_api.models.RequestSummarizedUsagesDetails(
        tenant_id=tenant_id,
        granularity=granularity,
        query_type='COST',
        group_by=['tenantName'],
        time_usage_started=time_usage_started.strftime('%Y-%m-%dT%H:%M:%SZ'),
        time_usage_ended=time_usage_ended.strftime('%Y-%m-%dT%H:%M:%SZ'),
        #compartment_depth=3,
    )

except Exception as e:
    print("\nException Error at 'usage_tenant' - " + str(e))
    

In [ ]:
######################################################################
#
# Make Request for usage data
# Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/client/oci.usage_api.UsageapiClient.html#oci.usage_api.UsageapiClient.request_summarized_usages
#
######################################################################

try:

    # Make a query to the Usage API using the deetails and filters defined in requestSummarizedUsagesDetails 
    request_summarized_usages = usage_client.request_summarized_usages(
        requestSummarizedUsagesDetails,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY
    )
    
    # Print the first element to visualize the structure of each response item
    print(request_summarized_usages.data.items[0])

except oci.exceptions.ServiceError as e:
    print("\nService Error at 'usage_tenant' - " + str(e))

except Exception as e:
    print("\nException Error at 'usage_tenant' - " + str(e))

In [ ]:
######################################################################
#
#   Verify the results obtained from the API by plotting them
#
######################################################################

def plot_usage_api_results(usages: oci.usage_api.models.UsageSummary):
    # Unpack the response object into different groups of data based on its tenancy
    tenancy_data = defaultdict(lambda: defaultdict(lambda: 0))

    for d in usages.data.items:
        tenancy_data[d.tenant_name][d.time_usage_started.strftime('%Y-%m-%dT%H:%M:%SZ')] += float(d.attributed_cost)

    # Create the stacked bar chart figure
    plt.figure(figsize=(12, 6))

    # Plot each category on top of the previous one
    bottom = None
    for tenancy, cost in tenancy_data.items():
        plt.bar( cost.keys(), cost.values(), bottom=bottom, label=tenancy)
        bottom = list(cost.values())

    # Adding title and labels
    plt.title('Daily Consumption by Tenancy')
    plt.xlabel('Date')
    plt.ylabel('Consumption (USD)')
    plt.legend()

    # Formatting the x-axis
    plt.xticks(rotation=45)

    # Show grid
    plt.grid(axis='y')

    # Display the chart
    plt.tight_layout()
    plt.show()

plot_usage_api_results(request_summarized_usages)

In [ ]:
######################################################################
#
# Create filter to limit results returned from query to only show cost for the 'Autonomous Data Warehouse' and 'Compute Services'.
#
# Available Dimensions for filtering are:
#   “service”, 
#   “skuName”, 
#   “skuPartNumber”, 
#   “unit”, 
#   “compartmentName”, 
#   “compartmentPath”, 
#   “compartmentId”, 
#   “platform”, 
#   “region”, 
#   “logicalAd”, 
#   “resourceId”, 
#   “tenantId”,
#   “tenantName”
#
# Dimension Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/models/oci.usage_api.models.Dimension.html#oci.usage_api.models.Dimension
# Filter Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/models/oci.usage_api.models.Filter.html#oci.usage_api.models.Filter
#
######################################################################

# Define rach dimension used for filtering separately
service_dem1 = Dimension(key='service', value='COMPUTE')
service_dem2 = Dimension(key='service', value='Autonomous Data Warehouse')

# You can combine dimension together when creating a filter element by passing them as a list and defining a logical operator. 
# All dimensions in the list will use the same operator.
service_filter = Filter(operator='OR', dimensions=[service_dem1, service_dem2])

# To create more complex filters create them separtely and then chain them together into a nested filter
service_filter1 = Filter(operator='AND', dimensions=[service_dem1])
service_filter2 = Filter(operator='AND', dimensions=[service_dem2])
service_filter = Filter(operator='OR', dimensions=[], filters=[service_filter1, service_filter2])

In [ ]:
######################################################################
#
# Make Request for usage data with the service filter defined above
# Docs @ https://docs.oracle.com/en-us/iaas/tools/python/2.136.0/api/usage_api/client/oci.usage_api.UsageapiClient.html#oci.usage_api.UsageapiClient.request_summarized_usages
#
######################################################################

try:

    # Set the filter for the request
    requestSummarizedUsagesDetails.filter = service_filter

    # Make a query to the Usage API using the deetails and filters defined in requestSummarizedUsagesDetails 
    request_summarized_usages = usage_client.request_summarized_usages(
        requestSummarizedUsagesDetails,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY
    )

except oci.exceptions.ServiceError as e:
    print("\nService Error at 'usage_tenant' - " + str(e))

except Exception as e:
    print("\nException Error at 'usage_tenant' - " + str(e))

In [ ]:
######################################################################
#
# Print the results from both tenancies after filtering based on the Compute and ADW services.
#
######################################################################

plot_usage_api_results(request_summarized_usages)